# Nayak, Anil Kumar
# 1001-396-015
# 2017-03-20
# Assignment_03_01

# This program is used for training phase of face detection 

In [12]:
import numpy as np
import json
import os
import cv2
import matplotlib.pyplot as plt
import operator
import timeit
import scipy.misc as sc

In [13]:
def create_rectangular_filters(N=24,min_max_size=10, max_max_size=16):
    """ This function uses the five base rectangular filters and
    creates all the possible different size and position filters
    inside an N by N area. The output of this function is a list of
    dictionaries. Each dictionary represents a rectangular filter
    at a particular position (x,y) and a particular width and Height.
    Note: the value of threshold for all the filters is set to zero.
    This value is just a place holder for use in task 2. The value
    of alpha for all the filters is set to 1. This is just a place
    holder to be used for Adaboost in task 2.
    Farhad Kamangar Feb. 13, 2017"""

    FeatureTypes = [(1, 2),(2, 1),(1, 3),(3, 1),(2, 2)]
    rectangular_filters = []
    for current_feature in FeatureTypes:
        for height in range(current_feature[0], N+1, current_feature[0]):
            for width in range(current_feature[1], N+1, current_feature[1]):
                #if max(height,width) >= min_max_size and max(height,width) <= max_max_size:
                for x in range(N - width+1):
                    for y in range(N - height+1):
                        temp_dict={"type":current_feature,"position":(x,y),"width":width,"height":height,"threshold":0,
                                   "alpha":1,"error":0,"selected":0,"pred_face":0,"pred_non_face":0,"number_face":0,
                                   "number_non_face":0,'miss_clasf_face':0,'miss_clasf_non_face':0}
                        rectangular_filters.append(temp_dict)
    return rectangular_filters

In [14]:
def define_min_max_threshold(w_c,h_c,m,n):
    pixels_in_section = (w_c / m) * (h_c / n)
    min_threshold = 0
    max_threshold = 0
    if (m == 2 and n == 1) or (m == 1 and n == 2):
        min_threshold = -pixels_in_section
        max_threshold = pixels_in_section
    elif (m == 3 and n == 1):
        min_threshold = -1 * 2 * pixels_in_section
        max_threshold = pixels_in_section
    elif (m == 1 and n == 3):
        min_threshold = -pixels_in_section
        max_threshold = 2 * pixels_in_section
    elif (m == 2 and n == 2):
        min_threshold = -1 * 2 * pixels_in_section
        max_threshold = 2 * pixels_in_section

    return min_threshold,max_threshold

In [15]:
def all_images_pixel(directory_name, convert_flag):
    pixel_all_image_in_directory = []
    all_image_names = os.listdir(directory_name)
    # data = np.array([cv2.imread(name) for name in os.listdir(directory_name+"/")], dtype=np.float64)
    # print(data(0))
    for image_name in all_image_names:
        image = cv2.imread(directory_name + "/" + image_name);

        if convert_flag == 1:
            image = cv2.resize(image, None, fx=24, fy=24, interpolation=cv2.INTER_LINEAR)

        gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        # print(gray_image.shape)
        images = {}
        images['image'] = gray_image / 255
        pixel_all_image_in_directory.append(images)

    return pixel_all_image_in_directory

In [16]:
def each_classifier_size_net_value(image, width, height, rows, cols):
    value = 0

    if rows == 1 and cols == 2:
        section1 = -np.sum(image[0: int(width / cols), 0:height])
        section2 = np.sum(image[int(width / cols):width, 0:height])
        value = section1 + section2
    elif rows == 2 and cols == 1:
        section1 = -np.sum(image[0:width, 0:int(height / rows)])
        section2 = np.sum(image[0:width, int(height / rows):height])
        value = section1 + section2
    elif rows == 3 and cols == 1:
        section1 = -np.sum(image[0:width, 0:int(height / 3)])
        section2 = np.sum(image[0:width, int(height / 3):int(height * (2 / 3))])
        section3 = -np.sum(image[0:width, int(height * (2 / 3)):height])
        value = section1 + section2 + section3
    elif rows == 1 and cols == 3:
        #print(width,height,cols,rows)
        section1 = np.sum(image[0:int(width / 3), 0:height])
        section2 = -np.sum(image[int(width / 3):int(width * (2 / 3)), 0:height])
        section3 = np.sum(image[int(width * (2 / 3)):width, 0:height])
        value = section1 + section2 + section3
    elif rows == 2 and cols == 2:
        section1 = -np.sum(image[0:int(width / 2), 0:int(height / 2)])
        section2 = np.sum(image[int(width / 2):width, 0:int(height / 2)])
        section3 = -np.sum(image[0:int(width / 2), int(height / 2):height])
        section4 = np.sum(image[int(width / 2):width, int(height / 2):height])
        value = section1 + section2 + section3 + section4

    return value
    

In [17]:
def write_json(classifers):
    with open('Nayak_03_02.json','w') as file:
        json.dump(classifers,file);

In [18]:
def calculate_alpha_for_classifier(error_rate_for_classifier):
    alpha_for_classifier = 0
    if error_rate_for_classifier != 0:
        alpha_for_classifier = (np.log(((1-error_rate_for_classifier)/error_rate_for_classifier)))/2
    return alpha_for_classifier

In [32]:
# Training Face Detection through adaboosting
# traing_flag = 1 if the classification will be trained for the actual training data
# traing_flag = -1 if the classification is carried out with the other than the actual data for misclassification


def train_adaboost_face_detection(faces, non_faces, list_of_rectrangle_templates, traing_flag,number_of_classifier_to_select):
    number_of_faces = np.size(faces)
    number_of_non_faces = np.size(non_faces)
    total_images = number_of_faces + number_of_non_faces

    y_faces = np.ones(number_of_faces)
    y_non_faces = np.zeros(number_of_non_faces)
    y_non_faces.fill(-1)

    weights_f = np.zeros(number_of_faces)
    weights_n_f = np.zeros(number_of_non_faces)
    weights_f.fill(1 / total_images)
    weights_n_f.fill(1 / total_images)

    strong_classifiers = []
    count_class = 0
    for i in range(number_of_classifier_to_select):
        list_of_rectrangle_templates.sort(key=operator.itemgetter('error'), reverse=False)

        if i >= 1:
            strong_classifier = {}
            # Select the least error classifier
            selected_classifier = list_of_rectrangle_templates[0]
            list_of_rectrangle_templates = list_of_rectrangle_templates[1:]

            print("Selected One Classifier", selected_classifier['type'],
                  selected_classifier['position'], selected_classifier['width'], selected_classifier['height'],
                  selected_classifier['alpha'], selected_classifier['threshold'])

            # calculate weight
            alpha = selected_classifier['alpha']
            pred_face = selected_classifier['pred_face']
            pred_non_face = selected_classifier['pred_non_face']
            
            e = weights_f * np.exp((-1*(y_faces * alpha * pred_face)))
            e1 = weights_n_f * np.exp((-1*(y_non_faces * alpha * pred_non_face)))

            weights_f = e / (np.sum(e) + np.sum(e1))
            weights_n_f = e1 / (np.sum(e) + np.sum(e1))

            strong_classifier['type'] = selected_classifier['type']
            strong_classifier['position'] = selected_classifier['position']
            strong_classifier['alpha'] = selected_classifier['alpha']
            strong_classifier['width'] = selected_classifier['width']
            strong_classifier['height'] = selected_classifier['height']
            strong_classifier['threshold'] = selected_classifier['threshold']
            strong_classifier['error'] = selected_classifier['error']
            strong_classifiers.append(strong_classifier)

        for classifier in list_of_rectrangle_templates:
            selected = classifier['selected']

            # Classifier is already selected
            if selected == 1:
                continue

            # "type":current_feature,"position":(x,y),"width":width,"height":height,"threshold":0,"alpha":1
            # print(classifier)
            # Classifier Type
            n, m = classifier['type']
            # position x and y
            x, y = classifier['position']
            # width w and height h
            w_c = classifier['width']
            h_c = classifier['height']
            # threshold
            # error
            # alpha


            if i == 0:
                count_class = count_class + 1
                # Take one classifier and visit all the images and find the values
                classifier['number_face'] = number_of_faces
                values_faces = np.zeros(number_of_faces)
                index = 0
                for face in faces:
                    image = face['image']
                    image_slice = image[x:x + w_c, y:y + h_c]
                    # print("Image Shape : ",image_slice.shape,x,y)
                    # accumulating all the values of one classifier for all the images
                    try:
                        value = each_classifier_size_net_value(image_slice, w_c, h_c, n, m)
                        values_faces[index] = value
                    except ValueError:
                        ''
                        # print("index : value ",index,value,w_c,h_c,n,m)
                    index = index + 1
                #print("values_faces",values_faces)
                # classifier['values_face'] = values_faces

                classifier['number_non_face'] = number_of_non_faces
                values_non_faces = np.zeros(number_of_non_faces)
                index = 0
                for non_face in non_faces:
                    image = non_face['image']
                    image_slice = image[x:x + w_c, y:y + h_c]
                    # print("Image Shape : ",image_slice.shape,x,y)
                    try:
                        # accumulating all the values of one classifier for all the images
                        value = each_classifier_size_net_value(image_slice, w_c, h_c, n, m)
                        values_non_faces[index] = value
                    except ValueError:
                        ''
                        # print("1 index : value ",index,value,w_c,h_c,n,m)
                    index = index + 1
                #print("values_non_faces",values_non_faces)
                # classifier['values_non_face'] = values_non_faces

                error = 0
                threshold = 0
                incorrect_face_classified = ''
                incorrect_non_face_classified = ''
                pred_face = ''
                pred_non_face = ''

                #print(w_c, h_c, m, n)
                min_threshold, max_threshold = define_min_max_threshold(w_c, h_c, m, n)
                #print(min_threshold, max_threshold)
                threshold_range = np.linspace(min_threshold, max_threshold, 2);
                #print(threshold_range)
                #error_per_th = np.zeros(20);
                #error_count = 0;
                for th in threshold_range:
                    correct_v_f = np.where(values_faces >= th, 1, -1)
                    correct_v_n_f = np.where(values_non_faces >= th, 1, -1)

                    incorrect_face_classify = np.where(y_faces != correct_v_f)
                    incorrect_non_face_classify = np.where(y_non_faces != correct_v_n_f)

                    incorrect_weights_f = np.take(weights_f, incorrect_face_classify)
                    incorrect_weights_n_f = np.take(weights_n_f, incorrect_non_face_classify)

                    total_incorrectly_classified_weights = sum(incorrect_weights_f[0]) + sum(incorrect_weights_n_f[0])
                    total_incorrectly_classified_weights = abs(total_incorrectly_classified_weights-0.5)

                    if error < total_incorrectly_classified_weights:
                        error = total_incorrectly_classified_weights
                        threshold = th
                        incorrect_face_classified = incorrect_face_classify
                        incorrect_non_face_classified = incorrect_non_face_classify
                        pred_face = correct_v_f
                        pred_non_face = correct_v_n_f

                classifier['error'] = error
                classifier['threshold'] = threshold
                classifier['miss_clasf_face'] = incorrect_face_classified
                classifier['miss_clasf_non_face'] = incorrect_non_face_classified
                classifier['pred_face'] = pred_face
                classifier['pred_non_face'] = pred_non_face

            else:
                incorrect_face_classify = classifier['miss_clasf_face']
                incorrect_non_face_classify = classifier['miss_clasf_non_face']

                incorrect_weights_f = np.take(weights_f, incorrect_face_classify)
                incorrect_weights_n_f = np.take(weights_n_f, incorrect_non_face_classify)

                total_incorrectly_classified_weights = sum(incorrect_weights_f[0]) + sum(incorrect_weights_n_f[0])

                classifier['error'] = total_incorrectly_classified_weights
                error = total_incorrectly_classified_weights

            classifier['alpha'] = calculate_alpha_for_classifier(error)

    return strong_classifiers

# Following will be Training Code Starting Point
1. train_face : Faces Directory Name 
2. train_non_face : Non Faces Directory Name

In [33]:
start = timeit.default_timer()

list_of_rectrangle_templates = create_rectangular_filters(24,10,16);
training_faces_sum_pixels = all_images_pixel("train_face",0)
training_non_faces_sum_pixels = all_images_pixel("train_non_face",0)

print("Number of Training Faces",np.size(training_faces_sum_pixels))
print("Number of Training Non Faces",np.size(training_non_faces_sum_pixels))

# Training Faces and Non Faces
strong_classifiers = train_adaboost_face_detection(training_faces_sum_pixels,training_non_faces_sum_pixels,list_of_rectrangle_templates,1,2)
stop = timeit.default_timer()

print(stop - start)
# Write Weak Hypothesis in  json file\
write_json(strong_classifiers)

Number of Training Faces 4
Number of Training Non Faces 4
Selected One Classifier (1, 2) (0, 0) 2 1 0 0
-----------


1
1
-----------


TypeError: ufunc 'multiply' did not contain a loop with signature matching types dtype('<U32') dtype('<U32') dtype('<U32')

# Following code will be for Testing 

In [ ]:
def read_images_pixel(directory_name):
    pixel_all_image_in_directory = []
    all_image_names = os.listdir(directory_name)

    for image_name in all_image_names:
        image = cv2.imread(directory_name + "/" + image_name);
        images = {}
        images['image'] = image
        images['name'] = image_name
        pixel_all_image_in_directory.append(images)

    return pixel_all_image_in_directory

In [ ]:
def detect_face(image,classifiers,scales):
    faces = []

    for scale in scales:
        image_face_detect = resize_image(image,scale);
        scale_factor = 100/scale
        gray_image = cv2.cvtColor(image_face_detect.copy(), cv2.COLOR_BGR2GRAY)
        normalized_image = gray_image / 255
        width, height = normalized_image.shape

        for h in range(0,height):
            for w in range(0,width):
                #validate weather the index out of bound does not occur
                if (w + 24 > width) or (h + 24 > height):
                    continue

                image_slice = normalized_image[w:w + 24, h:h + 24]
                Hx = 0
                for classifier in classifiers:
                    alpha = classifier['alpha']
                    threshold = classifier['threshold']
                    w_c = classifier['width']
                    h_c = classifier['height']
                    n, m = classifier['type']
                    x, y = classifier['position']

                    filter_window_slice = image_slice[x:x + w_c, y:y + h_c]
                    value = each_classifier_size_net_value(filter_window_slice, w_c, h_c, n, m)

                    h_x = 0;
                    if value >= threshold:
                        h_x = 1
                    else:
                        h_x = -1

                    Hx = Hx + (h_x*alpha)

                #find out that 24x24 window has face in it
                Hx_sign = np.sign(Hx)

                if Hx_sign == 1:
                    #print("Found Face",w,h)
                    face = [int(np.ceil(h*scale_factor)),int(np.ceil(w*scale_factor)),int(np.ceil(24*scale_factor)),int(np.ceil(24*scale_factor)),scale_factor]
                    faces.append(face)
                    w = w + 24

    return faces

In [ ]:
def resize_image(image,percentage):
    image_r = sc.imresize(image, percentage);
    return image_r

In [ ]:
def filter_redundant_faces(faces):
    filtered_faces = []
    flag = False

    for [x,y,h,w,scale] in faces:

        if len(filtered_faces) == 0:
            flag = True
        else:
            for [x1,y1,h1,w1,scale1] in filtered_faces:
                if x > x1-15 and x < x1+15:
                    flag = False
                    break
                else:
                    flag = True

        if flag == True:
            face = [x, y, h, w,scale]
            filtered_faces.append(face)

        flag = False

    return filtered_faces

In [ ]:
def draw_rectrangle(name,image,faces):
    for [x,y,h,w,scale] in faces:
        cv2.rectangle(image, (x,y), (x + w, y + h), (0, 0, 255), lineType=8, thickness=1);

    name = name + "_detected_face.jpg"
    cv2.imwrite(name, image)

In [ ]:
def read_json(file_name):
    with open(file_name) as file:
        classifiers = json.load(file)
        return classifiers

# Following code is starting point for testing

In [ ]:
# Testing Faces
file_name = "Nayak_03_02.json"
classifiers = read_json(file_name)
testing_images = read_images_pixel("Test")
scales = [100,75,50,25,10,8]
count = 0
for image in testing_images:
    start = timeit.default_timer()
    img = image['image']
    name = image['name']
    faces = detect_face(img,classifiers,scales)
    filtered_faces = filter_redundant_faces(faces)
    draw_rectrangle(name, img, filtered_faces)
    end = timeit.default_timer()
    print("Image ",count," Processing Complete in ",end - start," sec")
    count = count + 1

print("Face Detection Completed")
